# Классификация изображений

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback, EarlyStopping, CSVLogger
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
#from tensorflow.keras.layers import *
from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.losses import *
#from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split, StratifiedKFold

import random
import cv2
#from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
experiments = {"model_name": ["Model_1", "Model_2", "Model_3", "Model_4", "Model_5","Model_6", "Model_7", "Model_8", "Model_9", "Model_10", "Model_11", "Model_12","Model_13", "Model_14", "Model_15", "Model_16", "Model_17", "Model_18", "Model_19", "Model_20", "Model_21" ], 
               "accuracy": [92.74, 92.27,85.70, 81.41, 60.41, 79.69,85.40, 88.36,90.34, 76.99, 77.03, 76.34,76.60,76.94, 90.38, 75.96, 89.91, 92.31,91.41, 92.57, 97.08 ],
               "img_size": [224, 224, 224, 224, 224, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128],
               "augmentation_type":["ImageDataGenerator", "ImageDataGenerator", "Albumentations", "Albumentations", "Albumentations", "Albumentations", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator", "ImageDataGenerator"],
               "base_model":["Xception","FixEfficientNet-B7", "FixEfficientNet-B7", 'FixEfficientNet-B7', "FixEfficientNet-B7", "FixEfficientNet-B3","FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3", "FixEfficientNet-B3" ],
               "fine_tuning_frozen_layers":["no","no", "no", "no", "no", "no", "no", "no", "no", "no", "250", "100", "350", "300", "250", "250", "no", "no", "no", "no", "no"],
               "parameters_num_total": [21388594,64755873, 64755873,64755873, 64756897, 29041665, 29041665, 29041665, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017, 28534017],
               "parameters_num_trained": [21334066, 64445146, 64445146,64445146, 64445658, 28868410, 28868410, 28868410, 28361274, 28361274, 26940266, 28259134, 24473510, 26149130, 28361274, 28361274, 28361274, 28361274, 28361274, 28361274, 28361274 ],
               "parameters_num_not_trained": [54528, 310727, 310727,310727, 311239, 173255, 173255, 173255, 172743, 172743, 1593751, 274883, 4060507, 2384887, 172743, 172743, 172743, 172743, 172743, 172743, 172743 ],
               "layers_num_total": [136, 817,817, 817, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
               "layers_num_trained": [136, 817, 817, 817, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
               "trainable_variables**":["n/a","n/a","n/a","n/a","n/a","n/a","n/a","n/a",506, 506, 286, 419, 198, "?", "?", "?", 506, 506, 506, 506, 506],
               "batch_normalization": ["no","no", "no", "no", "yes", "yes", "yes", 'yes', "no", "no", "no", "no", "no", "no", "no", "no", "no", "no", "no", "no", "no"],
               "activation_hidden": ["relu","relu", "relu", "relu", "relu", "relu", "relu", "relu", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"],
               "activation_output": ["softmax","softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax", "softmax"],
               "callback_checkpoint": [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True],
               "callback_earlystop": [False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True],
               "learning_rate": [1e-4,1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, "ReduceLROnPlateau", 1e-4, "ReduceLROnPlateau", "1e-4 + ReduceLROnPlateau", "1e-4 + ReduceLROnPlateau", "1e-4 + ReduceLROnPlateau"],
               "epochs": [5,5,5, 25, 5, 10, 15, 25, 25, 25, 25, 25, 25, 25, 100, 25, 50, 50, 100, 100, 5],
               "tta": [False,False,False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]
               }

experiments_table = pd.DataFrame(experiments)
pd.set_option('display.width', 10000)
table = experiments_table.transpose()
table

План состоит в том чтобы итеративно изменять модель и вносить параметры в таблицу.

# Порядок экспериментов:
- **Model_1:** бейзлайн на основе базовой модели Xception
- **Model_2:** заменили base model на FixEfficientNet-B7, но точность стала даже ниже. Продолжим пока с этой моделью и попробуем изменить другие параметры.
- **Model_3:** заменили  Image Augmentation метод на Albumentations с различными параметрами, точность упала до 85.7%, но кажется что переобучение сильно сократилось.
- **Model_4:** добавили early stopping callback и увеличили количетсво эпох >> точность упала даже ниже
- **Model_5:** поменяли структуру модели на Sequential() и добавили BatchNormalization

На этом этапе были замечены несколько проблем, которые могли повлечь за собой такой низкий accuracy score. 
Изменения на этой версии:
- убираем аугментацию тестовой выборки
- save_best_only = True
- также уменьшаем размер изображения на этих тестовых итерациях до 128 чтобы все быстрее считалось и увеличиваем количество эпох до 10
- также кажется что на этом этапе лучше сменить base_model чтобы обучение проходило быстрее и дальше докрутить какую-то из моделей когда вернемся хотя бы к изначальному значению точности. FixEfficientNet-B7 использует 66М параметров, заменим его на FixEfficientNet-B5, который использует в два раза меньше (30М).


- **Model_6:** со всеми упомянутыми выше шагами Accuracy rate выросла обратно до 79.69%. Следующее что можно попробовать - убрать слишком большее количество аугментаций, которые мы используем. Насколько я поняла в теории, на меньших количествах эпох должно лучше сработать если мы используем меньше аугментаций. Вместо Albumentations будем использовать стандартные аугментации из ImageDataGenerator.

- **Model_7:** из аугментаций оставим только:
    -  horizontal_flip = 0.3,
    -  rotation_range = 10,
    -  zoom_range = 0.1,
  и увеличиваем количество эпох до 15. >> В результате мы хотя бы вернулись к уровню модели 3, но она все еще далека от изначальной.

- **Model_8:** увеличим количество эпох и чуть-чуть поправим параметры аугментации:
   -  horizontal_flip = True,
   -  width_shift_range = 0.1,
   -  height_shift_range = 0.1,
   -  zoom_range = [0.75, 1.25],

- **Model_9:** уберем первый dense слой и batch normalization >> Accuracy увеличилась до 90.34%.

- **Model_10:** попробуем понизить learning rate до 1e-5. Несмотря на то что точность снова ухудшилась до 76.99%, кажется что learning rate подобрана неплохо и мы можем взять эту модель за базовую и просто добавить большее количество эпох. 

- **Model_11:** попробуем перейти к тонкой настройке base model. Пробуем заморозить сеть и ее веса и для начала заморозим половину слоев. >> Получили accurcacy 77.03%. Если наш план сначала натренировать модель на 25-ти эпохах, а потом когда подберем лучший вариант добавить максимальное количество эпох, стоит ли полагаться на то что точность немного улучшилась на первых 25-ти эпохах?

- **Model_12:** заморозим четверть весов, но пока не будем уменьшать learning rate до 1e-6.

- **Model_13:** наоборот оставим размороженной четверть весов, а остальные три четверти заморозим.

- **Model_14:** зморозим 300 первых слоев. >> Точность все равно немного меньше чем в случае 250 слоев, поэтому вернемся к этому параметру

- **Model_15:** итак, мы готовы попробовать модель на 100 эпохах! Если результат будет не очень, вернемся к Модели_9.  >> Результат получился 90.38%,что на небольшую долю лучше чем Модель 9, но все же мы потратили слишком много времени на это по сравнению с Моделью_9.

- **Model_16:** вообще я вспомнила что в модели 10, на которой точность ухудшилась мы уменьшили learning rate, может быть если поэксперементировать с другими learning rate можно добиться лучшего результата. Попробуем [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/) и вернемся к 25 эпохам для теста. >> Точность продолжает оставаться довольно низкой (75.96%)

- **Model_17:** остается вопрос что лучше - Модель_9, которая показала точность 90% на 25ти эпохах, или надеяться что последняя модель если увеличить количество эпох покажет результат лучше? Думаю вернуться к Модели_9 и посмотреть что будет, если прогнать ее на 50ти эпохах. >> Получилась accuracy 89.91%, что даже немного меньше чем изначальная Модель_9.

- **Model_18:** кажется, что нужно вернуться к предыдущим настройкам без заморозки части сети

- **Model_19:** все тоже самое, но изменяем параметры в ReduceOnPlateau (эксперементируем с длительностью ожидания и фактором на который должна падать learning rate), добавляем TTA и увеличиваем количество эпох до 100.
ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=8, min_lr=1e-5, verbose=1)


- **Model_20:** увеличили размер картинки до 500 и поставила 100 эпох. После 9ти часов ноутбук остановился :( 

- **Model_21:** все тоже самое но только 5 эпох, чтобы успеть в недельную квоту. При таком количестве эпох никакие оптимизационные вещи по learning rate ReduceOnPlateau и тд просто не успевают срабатывать, и кажется единственное что сильно помогло увеличить качество модели это увеличение изображения. 


In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 5  # эпох на обучение
BATCH_SIZE           = 8 # (16)уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 500 # (224)какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/" # рабочая директория

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

### Аугментация данных

In [ ]:
#transform = A.Compose([
#    A.Blur(blur_limit=3),
#    A.ChannelDropout (p=0.1),
#    A.CoarseDropout (p=0.1),
#    A.Downscale (p=0.2),
#    A.FancyPCA(p=0.2),
#    A.Flip(),
#    A.HorizontalFlip(p=0.5),
#    A.HueSaturationValue(p=0.5),
#    A.GaussianBlur(p=0.05),
#    A.GaussNoise(p=0.2),
#    A.OpticalDistortion(),
#    A.RandomBrightnessContrast(p=0.5),
#    A.RandomFog(p=0.2),
#    A.RandomShadow(p=0.2),
#    A.Rotate(limit=30, 
#             interpolation=1, 
#             border_mode=4, 
#             value=None, 
#             mask_value=None, 
#             always_apply=False, 
#             p=0.5),
#    A.ShiftScaleRotate(shift_limit=0.0625, 
#                       scale_limit=0.5,  
#                       rotate_limit=45, 
#                       p=0.75),
#])

In [ ]:
#train_datagen = ImageDataAugmentor(
#        rescale=1./255,
#        augment = transform,
#        validation_split=VAL_SPLIT
#        )

train_datagen = ImageDataGenerator(
        #rescale=1./255,
        horizontal_flip = True,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        rotation_range = 10,
        zoom_range = [0.75, 1.25], #0.1
        #brightness_range=(0.5 , 1.5)
        validation_split=VAL_SPLIT
        )


test_datagen = ImageDataGenerator(
        #rescale=1./255,
        validation_split=VAL_SPLIT
        )


sub_datagen = ImageDataGenerator(
        #rescale=1./255,
        horizontal_flip = True,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        rotation_range = 10,
        zoom_range = [0.75, 1.25]
        )

### Генерация данных

In [ ]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = test_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

sub_generator = sub_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)

In [ ]:
# from skimage import io

# def imshow(image_RGB):
#     io.imshow(image_RGB)
#     io.show()
    
    
# x,y = train_generator.next()
# print("Example")
# plt.figure(figsize=(12,8))

# for i in range(0,9):
#     image = x[i]
#     plt.subplot(3,3, i+1)
#     plt.imshow(image)
# plt.show()

# Построение модели

### Загружаем предобученную сеть:

In [ ]:
base_model = tf.keras.applications.EfficientNetB5(weights='imagenet', include_top=False, input_shape = input_shape)

In [ ]:
#first train only top layers (which were randomly initialized)
#base_model.trainable = False

In [ ]:
# Устанавливаем новую "голову" (head)
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    #Dense(256, activation='relu'),
    #BatchNormalization(),
    Dense(CLASS_NUM, activation='softmax')
])

# this is the model we will train
#model = Model(inputs=base_model.input, outputs=predictions)
model.compile(
    loss="categorical_crossentropy", 
    optimizer=optimizers.Adam(lr=LR),
    #optimizer=optimizers.Adam(learning_Rate=ReduceLROnPlateau, ),
    metrics=["accuracy"])

#model = Sequential([
#  EfficientNetB0(weights='imagenet', input_shape=(*image_size, 3), include_top=False), #предобученная нейросеть из модуля keras.applications
#  GlobalMaxPool2D(),
#  Dropout(0.5),
#  Dense(10)
#])

#model.compile(
#    loss=CategoricalCrossentropy(from_logits=True),
#    optimizer=Adam(ExponentialDecay(1e-3, 100, 0.9)),
#    metrics='accuracy'
#)

In [ ]:
#base_model.trainable = True
## finetune from this layer onwards:
#fine_tune_at=250

## freeze all the layers before the fine_tune_at_layer:
#for layer in base_model.layers[:fine_tune_at]:
#    layer.trainable=False

In [ ]:
# checking the trainable status of the individual layers of the model:
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
# checking the trainable status of the individual layers of the base model:
#for layer in base_model.layers:
#    print(layer, layer.trainable)

In [ ]:
model.summary()

In [ ]:
print(len(base_model.layers))
print(len(model.layers))
print(len(base_model.trainable_variables))
print(len(model.trainable_variables))

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5',
                             monitor = 'val_accuracy', 
                             verbose = 1, 
                             mode = 'max',
                             save_best_only=True
                            )
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=1e-5, verbose=1)
earlystop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop, reduce_lr]


# Рекомендация 1. Добавьте другие функции из https://keras.io/callbacks/
# Рекомендация 2. Используйте разные техники управления Learning Rate
# https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng)
# http://teleported.in/posts/cyclic-learning-rate/ (eng)

Обучаем:

In [ ]:
train_generator

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели составила 97.08%. 
Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

- Model_1: мы видим значительное переобучение
- Model_2: даже большее переобучение на шаге 1-4
- Model_3: из-за добавления аугментированных данных мы избавились от переобучения, но accuracy стал даже ниже
- Model_4: модель плохо показала себя на валидационной выборке, качество сильно скачет между эпохами. возможно это можно исправить тем что мы уменьшим learning rate или это из-за того что мы аугментируем тестовую выборку тоже
- Model_5: начиная с 3 итерации все очень сильно переобучилось
- Model_6: начиная с 2 итерации есть переобучение
- Model_7: все еще есть переобучение
- Model_8: все еще есть переобучение, но показатели тестовой и валидационной выборки начинают сближаться и хотя бы нет гигантских колебаний.
- Model_9: кажется что скачки точности валидационной выборки после пятой эпохи могут быть сглажены если мы уменьшим learning rate?
- Model_10: наконец-то кривая валидационной выборки стала ровной и к 25ой эпохе обе кривые очень близки друг к другу и очень плавно повышаются. кажется уменьшение learning rate помогло и теперь нужно только увеличить количество эпох.
- Model_11: по графику не кажется что что-то изменилось, возможно граф стал чуть более пологим
- Model_12: начиная с 5 и по 20 эпоху валидационная выборка показывает даже выше точность чем тестовая.
- Model_13: на графике нет особых изменений
- Model_14: валидационная кривая чуточку лучше чем на предыдущем графике
- Model_15: начиная с 25о эпохи, модель начала переобучаться и параметр точности вышел в плато, то есть эта модель не позволяет нам особо повысить точность, хотя мы и достигли 90.38%.
- Model_16: тестовые и валидационные кривые очень близки и у них достаточный наклон чтобы предполагать, что при увеличении эпох результат также улучшится
- Model_17: модель остановилась где-то на 35 эпохе, потому что не было улучшения, кривая опять "прыгает" скорее всего из-за слишком большого learning rate
- Model_18: мы сразу переобучаемся и хотя и приближение валидационной точности сглаживается уменьшением learning rate, теперь у нас есть проблема переобучения
- Model_19:  видимо learning rate затухает слишком быстро, так как после нескольких эпох мы больше не приближаемся особо к трейн точности
- Model_20: уже лучше, но кажется что можно дать еще больше эпох для ReduceOnPlateu
- Model_21: график валидационной кривой достаточно пологий что оставляет мало шансов на то что ситуация сильно улучшилась бы с большим количеством эпох, с другой стороны качество хорошее начиная прямо с 1ой эпохи.

# Предсказание на тестовых данных

In [ ]:
sub_generator.samples

In [ ]:
sub_generator.reset()
predictions = model.predict_generator(sub_generator, steps=len(sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

# Рекомендация: попробуйте добавить Test Time Augmentation (TTA)
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [ ]:
submission.head()

# TTA

In [ ]:
# sub_generator.reset()
# predictions_tta = []
# tta_steps = 10

# for _ in range(tta_steps):
#     predictions_tta.append(model.predict(sub_generator, verbose=1))
#     sub_generator.reset()
    
# predictions_tta = np.mean(np.array(predictions_tta), axis=0).argmax(axis=1)

# submission_tta = pd.DataFrame({
#     'Id': sub_generator.filenames,
#     'Category': predictions_tta
#     }, columns=['Id', 'Category'])

# submission_tta.to_csv('submission_tta.csv', index=False)

In [ ]:
# scores_tta = model.evaluate_generator(sub_generator, steps=len(sub_generator), verbose=1)
# print("Accuracy: %.2f%%" % (scores_tta[1]*100))

In [ ]:
# model_2 = load_model('last_model.hdf5')
# sub_generator.reset()
# predictions_tta_ansemble = []
# for _ in range(EPOCHS):
#     predictions_tta_ansemble.append(0.6*model.predict(sub_generator, verbose=1) \
#                                     + 0.2*model_2.predict(sub_generator, verbose=1))
#     sub_generator.reset()
# predictions_tta_ansemble = np.mean(np.array(predictions_tta_ansemble), axis=0).argmax(axis=1)
# submission_tta_ansemble = pd.DataFrame({
#     'Id': sub_generator.filenames,
#     'Category': predictions_tta_ansemble
# }, columns=['Id', 'Category'])
# submission_tta_ansemble.to_csv('submission_tta_ansemble.csv', index=False)

In [ ]:
# tta_steps=10
# predictions = []

# for i in range(tta_steps):
#     preds = model.predict_generator(test_generator, verbose=1)
#     predictions.append(preds)
    
# pred = np.mean(predicitons, axis=0)

In [ ]:
# predictions = np.argmax(pred, axis=-1)
# label_map = (train_generator.class_indices)
# label_map = dict((v,k) for k,v in label_map.items())
# predictions = [label_map[k] for k in predictions]
# filenames_with_dir = test_generator.filenames
# submission = pd.DataFrme({'Predict': predictions}, columns = ['Predict'], index = filenames_with_dir)
# tmp_y = pd.concat([submission['Predict'], test_files['Category']], axis=1, sort=False)

In [ ]:
# # Clean PATH
# import shutil
# shutil.rmtree(PATH)

# Что еще можно было попробовать сделать:

* попробовать другую базовую модель, более новую или специализированную
* Попробовать ансамбли моделей
* Доработать ТТА
* Дополнительно*: Используйте разные техники управления Learning Rate (https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng) http://teleported.in/posts/cyclic-learning-rate/ (eng))
* Дополнительно*: Добавьте более продвинутые библиотеки аугментации изображений (например, Albumentations )